# Fase 03: Optimización del modelo en HAR (Hailo Archive)

In [1]:
# Imports

import json
import os
import gc

import numpy as np
import tensorflow as tf
from IPython.display import SVG
from matplotlib import patches
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.python.eager.context import eager_mode
import torch
import onnx
import onnxruntime as ort
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from itertools import islice

# import the hailo sdk client relevant classes
from hailo_sdk_client import ClientRunner, InferenceContext

# Funciones y parámetros de la CNN base
import sys
sys.path.append('../src/models/')

from cnn21_pix import *

2025-04-18 03:16:03.168181: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 03:16:03.200332: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-18 03:16:03.201050: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-18 03:16:03.789778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 1. Optimización rápida

### 1.1. Cargar el dataset

In [2]:
# Configuración de parámetros

cuda=True if torch.cuda.is_available() else False
# device=torch.device('cuda' if cuda else 'cpu')
device='cpu'

In [3]:
# Carga de datos

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'

(datos,H,V,B)=read_raw(DATASET)
(truth,H1,V1)=read_pgm(GT)

# Durante la ejecucion de la red vamos a coger patches de tamano cuadrado
sizex=32; sizey=32 

# Hacemos padding en el dataset para poder aprovechar hasta el borde
if(PAD):
    datos=torch.FloatTensor(np.pad(datos,((sizey//2,sizey//2),(sizex//2,sizex//2),(0,0)),'symmetric'))
    H=H+2*(sizex//2); V=V+2*(sizey//2)
    truth=np.reshape(truth,(-1,H1))
    truth=np.pad(truth,((sizey//2,sizey//2),(sizex//2,sizex//2)),'constant')
    H1=H1+2*(sizex//2); V1=V1+2*(sizey//2)
    truth=np.reshape(truth,(H1*V1))
    
# Necesitamos los datos en band-vector para hacer convoluciones
datos=np.transpose(datos,(2,0,1))

# Extraer parches para la calibración
num_patches = 100000
calib_dataset = np.zeros((num_patches, sizex, sizey, B), dtype=np.float32)

# Seleccionar muestras aleatorias para la calibración
samples_y = np.random.randint(sizey//2, V - sizey//2, size=(num_patches,))
samples_x = np.random.randint(sizex//2, H - sizex//2, size=(num_patches,))

for idx in range(num_patches):
    x, y = samples_x[idx], samples_y[idx]
    patch = select_patch(datos, sizex, sizey, x, y) # Extraer parche con formato (B, H, V)
    calib_dataset[idx, :, :, :] = np.transpose(patch, (1,2,0)) # Convertir a (H, V, B)

* Read dataset: ../data/imagenes_rios/oitaven_river.raw
  B: 5 H: 6722 V: 6689
  Read: 224817290
* Read GT: ../data/imagenes_rios/oitaven_river.pgm
  H: 6722 V: 6689 depth: 10
  Read: 44963458


### 1.2. Optimizar el modelo en HAR

In [32]:
# Cargamos el modelo HAR convertido

model_name = "../results/model_cnn21"
hailo_model_har_name = f"{model_name}_hailo_model.har"
assert os.path.isfile(hailo_model_har_name), "Please provide valid path for HAR file"
runner = ClientRunner(har=hailo_model_har_name, hw_arch="hailo8l")

In [7]:
# Cuantización del modelo

# Now we will create a model script, that tells the compiler to add a normalization on the beginning
# of the model (that is why we didn't normalize the calibration set;
# Otherwise we would have to normalize it before using it)
# Batch size is 8 by default
# alls = "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n"
alls = ""

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script(alls)

# Call Optimize to perform the optimization process
runner.optimize(calib_dataset)

# Save the result state to a Quantized HAR file
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

[warning] Model script is empty
[info] Loading model script commands to model_cnn21 from string
[warning] Model script is empty
[info] Starting Model Optimization
[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[warning] Running model optimization with zero level of optimization is not recommended for production use and might lead to suboptimal accuracy results
[info] Model received quantization params from the hn


2025-03-24 23:03:38.726546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NOT_INITIALIZED: initialization error
2025-03-24 23:03:38.726588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: debian
2025-03-24 23:03:38.726595: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: debian
2025-03-24 23:03:38.726706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 570.124.6
2025-03-24 23:03:38.726730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 570.124.6
2025-03-24 23:03:38.726736: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 570.124.6
2025-03-24 23:03:38.749696: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS 

[info] MatmulDecompose skipped
[info] Starting Mixed Precision
[info] Model Optimization Algorithm Mixed Precision is done (completion time is 00:00:00.02)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration


Calibration:   0%|                                          | 0/64 [00:00<?, ?entries/s]2025-03-24 23:03:39.674519: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-03-24 23:03:39.702053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,32,32,5]
	 [[{{node Placeholder/_0}}]]
Calibration:   2%|▌                                 | 1/64 [00:02<03:07,  2.98s/entries]2025-03-24 23:03:42.191012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not ind

Calibration:  89%|█████████████████████████████▍   | 57/64 [00:03<00:00, 32.25entries/s]2025-03-24 23:03:42.499132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,32,32,5]
	 [[{{node Placeholder/_0}}]]
2025-03-24 23:03:42.505303: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,32,32,5]
	 [[{{node Placeholder/_0}}]]
2025-03-24 23:03:42.511417: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT:

[info] Model Optimization Algorithm Statistics Collector is done (completion time is 00:00:03.46)
[info] Starting Fix zp_comp Encoding
[info] Model Optimization Algorithm Fix zp_comp Encoding is done (completion time is 00:00:00.00)
[info] Matmul Equalization skipped
[info] No shifts available for layer model_cnn21/conv1/conv_op, using max shift instead. delta=11.2811
[info] No shifts available for layer model_cnn21/conv2/conv_op, using max shift instead. delta=0.0792
[info] No shifts available for layer model_cnn21/conv2/conv_op, using max shift instead. delta=0.0396
[info] No shifts available for layer model_cnn21/conv1/conv_op, using max shift instead. delta=11.2811
[info] Finetune encoding skipped
[info] Bias Correction skipped
[info] Adaround skipped
[info] Quantization-Aware Fine-Tuning skipped
[info] Layer Noise Analysis skipped
[info] Model Optimization is done
[info] Saved HAR to: /home/pablo/Documentos/Enxeñaría Informática/Cuarto/2º Cuatrimestre/Traballo Fin de Grao/Proyecto

## 2. Optimización detallada

### 2.1. Carga de datos

In [2]:
# Definir parámetros

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'

# Queremos usar todos los datos para la inferencia
SAMPLES=[0,0]
PAD=1
AUM=0

In [3]:
# Carga de datos

(datos,H,V,B)=read_raw(DATASET)
(truth,H1,V1)=read_pgm(GT)

# Durante la ejecucion de la red vamos a coger patches de tamano cuadrado
sizex=32; sizey=32 

# Hacemos padding en el dataset para poder aprovechar hasta el borde
if(PAD):
    datos=torch.FloatTensor(np.pad(datos,((sizey//2,sizey//2),(sizex//2,sizex//2),(0,0)),'symmetric'))
    H=H+2*(sizex//2); V=V+2*(sizey//2)
    truth=np.reshape(truth,(-1,H1))
    truth=np.pad(truth,((sizey//2,sizey//2),(sizex//2,sizex//2)),'constant')
    H1=H1+2*(sizex//2); V1=V1+2*(sizey//2)
    truth=np.reshape(truth,(H1*V1))
    
# Necesitamos los datos en band-vector para hacer convoluciones
datos=np.transpose(datos,(2,0,1))

# Seleccionar conjunto de test (en este caso es una predicción)
(train,val,test,nclases,nclases_no_vacias)=select_training_samples(truth,H,V,sizex,sizey,SAMPLES)
dataset_test=HyperDataset(datos,truth,test,H,V,sizex,sizey)
print('  - test dataset:',len(dataset_test))

# Dataloader
batch_size=100 # defecto 100
# Indicamos shuffle = True porque no se evalúa todo el dataset
test_loader=DataLoader(dataset_test,batch_size,shuffle=True)

* Read dataset: ../data/imagenes_rios/oitaven_river.raw
  B: 5 H: 6722 V: 6689
  Read: 224817290
* Read GT: ../data/imagenes_rios/oitaven_river.pgm
  H: 6722 V: 6689 depth: 10
  Read: 44963458
* Select training samples
  nclasses: 10
  Class  # :   total | train |   val |    test
  Class  1 :  309248 |     0 |     0 |  309248
  Class  2 :  113324 |     0 |     0 |  113324
  Class  3 :   79152 |     0 |     0 |   79152
  Class  4 :   43861 |     0 |     0 |   43861
  Class  5 :  128022 |     0 |     0 |  128022
  Class  6 :   78785 |     0 |     0 |   78785
  Class  7 : 2428482 |     0 |     0 | 2428482
  Class  8 : 1829360 |     0 |     0 | 1829360
  Class  9 :  193884 |     0 |     0 |  193884
  Class 10 :  863061 |     0 |     0 |  863061
  - test dataset: 6067179


### 2.2 Evaluar el modelo en har sin optimización

In [4]:
# Cargar el modelo convertido a ONNX

ort_session = ort.InferenceSession("../results/model_cnn21.onnx")

# Cargar el modelo en HAR

model_name = "../results/model_cnn21"
hailo_model_har_name = f"{model_name}_hailo_model.har"
assert os.path.isfile(hailo_model_har_name), "Please provide valid path for HAR file"
runner = ClientRunner(har=hailo_model_har_name, hw_arch="hailo8l")

In [5]:
# Comprobacion rapida de la diferencia de precisión

# Crear tensor de entrada de prueba
input_tensor = torch.randn(1, B, sizex, sizey).to("cpu")
input_np = input_tensor.cpu().numpy()

# Realizar la inferencia en ONNX
output_onnx = ort_session.run(None, {'input': input_np})[0]

# print(output_onnx)

# Realizar la inferencia en HAR
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    input_har = np.transpose(input_np, (0, 2, 3, 1))

    # Realizar la inferencia en el modelo .har
    output_har = runner.infer(ctx, input_har)

    # print(output_har) 

# Comparar las diferencias
error = np.abs(output_onnx - output_har).mean()
print(f'Error medio entre PyTorch y ONNX: {error}')

2025-04-18 03:16:53.739187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-18 03:16:53.741036: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Inference: 0entries [00:00, ?entries/s]2025-04-18 03:16:54.018700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must f

Error medio entre PyTorch y ONNX: 1.52587890625e-05


In [6]:
'''
# El bucle de inferencia tiene un memory leak que proviene del SDK de hailo con el emulador NATIVE
# Se realiza la evaluacion, por tanto, con un unico batch

# Inferencia

output=np.zeros(H*V,dtype=np.uint8)

# Medir el tiempo de ejecución
start_time = time.time()

# Realizar la predicción
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    total=0
    for (inputs, labels) in test_loader:
        # Convertir inputs a un formato adecuado para Hailo (numpy array)
        # Cambiamos de (batch_size, B, sizex, sizey) a (batch_size, sizey, sizex, B)
        inputs_np = np.transpose(inputs.numpy(), (0, 3, 2, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_np)
        
        predicted=np.argmax(native_res, axis=-1)
        predicted=predicted.squeeze()
        
        # Asignar las predicciones al array de salida
        for i in range(len(predicted)):
            output[test[total+i]]=np.uint8(predicted[i]+1)
        total+=labels.size(0)

        # Mostrar el progreso
        if(total%100000==0): print('  Test:',total,'/',len(dataset_test))
        
end_time = time.time()

print("Prediction time: {:.4f} seconds".format(end_time - start_time))

'''

'\n# El bucle de inferencia tiene un memory leak que proviene del SDK de hailo con el emulador NATIVE\n# Se realiza la evaluacion, por tanto, con un unico batch\n\n# Inferencia\n\noutput=np.zeros(H*V,dtype=np.uint8)\n\n# Medir el tiempo de ejecución\nstart_time = time.time()\n\n# Realizar la predicción\nwith runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:\n    total=0\n    for (inputs, labels) in test_loader:\n        # Convertir inputs a un formato adecuado para Hailo (numpy array)\n        # Cambiamos de (batch_size, B, sizex, sizey) a (batch_size, sizey, sizex, B)\n        inputs_np = np.transpose(inputs.numpy(), (0, 3, 2, 1))\n\n        # Realizar la inferencia en el modelo .har\n        native_res = runner.infer(ctx, inputs_np)\n        \n        predicted=np.argmax(native_res, axis=-1)\n        predicted=predicted.squeeze()\n        \n        # Asignar las predicciones al array de salida\n        for i in range(len(predicted)):\n            output[test[total+i]]=np.uint

In [7]:
# Evaluar el modelo en HAR sin optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:18.366492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1540.91entries/s]


[Iteración 1] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:18.861796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2050.92entries/s]


[Iteración 2] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:19.306366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2172.28entries/s]


...

[Iteración 98] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:02.892880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2248.20entries/s]


[Iteración 99] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:03.326304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2316.19entries/s]

[Iteración 100] Error medio: 0.0000 ; Coincidencias: 100/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.000000
Precisión global: 100.00%


### 2.3 Aplicar modificaciones de optimización al modelo y evaluarlo

In [8]:
# Crear un model script para el proceso de optimización

model_script_lines = [
    # Add normalization layer with mean [123.675, 116.28, 103.53] and std [58.395, 57.12, 57.375])
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # For multiple input nodes:
    # {normalization_layer_name_1} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_1_from_hn})\n',
    # {normalization_layer_name_2} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_2_from_hn})\n',
    # ...
]

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script("".join(model_script_lines))
runner.optimize_full_precision()

[warning] Model script is empty
[info] Loading model script commands to model_cnn21 from string
[warning] Model script is empty
[warning] DEPRECATION WARNING: Optimizing in full precision will require calibration data in the near future, to allow more accurate optimization algorithms which require inference on actual data.


In [9]:
# Evaluar el modelo en HAR con optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_FP_OPTIMIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:34.587670: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 270.39entries/s]


[Iteración 1] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:35.459731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2039.36entries/s]


[Iteración 2] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:35.912833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2168.35entries/s]


...

[Iteración 98] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:19:19.253537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2241.04entries/s]


[Iteración 99] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:19:19.710452: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2302.29entries/s]

[Iteración 100] Error medio: 0.0000 ; Coincidencias: 100/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.000000
Precisión global: 100.00%


### 2.4 Cuantizar el modelo y evaluarlo

In [10]:
# Crear un dataset de calibración
# The original images are being used, just as the input to the SDK_FP_OPTIMIZED emulator
total_images = 10000

calib_dataset = np.zeros((total_images, sizex, sizey, B), dtype = np.float32)

count = 0

# Seleccionamos batches del dataloader para guardarlos en el dataset
for inputs, _ in test_loader:
    for img in inputs:
        if count >= total_images:
            break
            
        # Los inputs son de la forma (batch_size, B, sizex, sizey)
        img_np = img.numpy()
        # Trasponemos los inputs a formato (batch_size, sizex, sizey, B)
        img_har = np.transpose(img_np, (1, 2, 0))

        calib_dataset[count] = img_har
        count += 1
        
    if count >= total_images:
        break

In [11]:
# Cuantizar el modelo con el dataset de calibración

# For calling Optimize, use the short version: runner.optimize(calib_dataset)
# A more general approach is being used here that works also with multiple input nodes.
# The calibration dataset could also be a dictionary with the format:
# {input_layer_name_1_from_hn: layer_1_calib_dataset, input_layer_name_2_from_hn: layer_2_calib_dataset}
hn_layers = runner.get_hn_dict()["layers"]
print("Input layers are: ")
print([layer for layer in hn_layers if hn_layers[layer]["type"] == "input_layer"])  # See available input layer names
calib_dataset_dict = {"model_cnn21/input_layer1": calib_dataset}  # In our case there is only one input layer
runner.optimize(calib_dataset_dict)

Input layers are: 
['model_cnn21/input_layer1']
[info] Starting Model Optimization
[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[warning] Running model optimization with zero level of optimization is not recommended for production use and might lead to suboptimal accuracy results
[info] Model received quantization params from the hn
[info] MatmulDecompose skipped
[info] Starting Mixed Precision
[info] Model Optimization Algorithm Mixed Precision is done (completion time is 00:00:00.02)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration


Calibration:   0%|                                          | 0/64 [00:00<?, ?entries/s]2025-04-18 03:19:32.341714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-04-18 03:19:32.362040: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,32,32,5]
	 [[{{node Placeholder/_0}}]]
Calibration:   2%|▌                                 | 1/64 [00:01<01:30,  1.43s/entries]2025-04-18 03:19:33.177011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not ind

Calibration:  59%|███████████████████▌             | 38/64 [00:01<00:00, 35.94entries/s]2025-04-18 03:19:33.381743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,32,32,5]
	 [[{{node Placeholder/_0}}]]
2025-04-18 03:19:33.387563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,32,32,5]
	 [[{{node Placeholder/_0}}]]
2025-04-18 03:19:33.393277: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT:

Calibration: 100%|█████████████████████████████████| 64/64 [00:01<00:00, 35.57entries/s]


[info] Model Optimization Algorithm Statistics Collector is done (completion time is 00:00:01.91)
[info] Starting Fix zp_comp Encoding
[info] Model Optimization Algorithm Fix zp_comp Encoding is done (completion time is 00:00:00.00)
[info] Matmul Equalization skipped
[info] Starting MatmulDecomposeFix
[info] Model Optimization Algorithm MatmulDecomposeFix is done (completion time is 00:00:00.00)
[info] Finetune encoding skipped
[info] Bias Correction skipped
[info] Adaround skipped
[info] Quantization-Aware Fine-Tuning skipped
[info] Layer Noise Analysis skipped
[info] Model Optimization is done


In [12]:
# Evaluar el modelo en HAR cuantizado

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_QUANTIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

Inference: 0entries [00:00, ?entries/s]2025-04-18 03:20:57.430008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:01, 52.95entries/s]


[Iteración 1] Error medio: 0.0400 ; Coincidencias: 98/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:20:59.809331: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1062.15entries/s]


[Iteración 2] Error medio: 0.0300 ; Coincidencias: 97/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:21:00.317622: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1005.83entries/s]


...

[Iteración 98] Error medio: 0.0300 ; Coincidencias: 98/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:21:50.602947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1223.45entries/s]


[Iteración 99] Error medio: 0.0100 ; Coincidencias: 99/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:21:51.081860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1227.23entries/s]

[Iteración 100] Error medio: 0.1400 ; Coincidencias: 94/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.048600
Precisión global: 97.42%


In [13]:
# Guardar el modelo cuantizado
# Let's save the runner's state to a Quantized HAR
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

[info] Saved HAR to: /home/pablo/Documentos/Enxeñaría Informática/Cuarto/2º Cuatrimestre/Traballo Fin de Grao/Proyecto/results/model_cnn21_quantized_model.har
